Task 2: Data Cleaning & Preprocessing

In [49]:
import pandas as pd
import os
import re

In [50]:
def data_cleaning_and_preprocessing(categories, base_path= ".", save_path="cleaned_data.csv"):
    files = []

    # Merge on parent asin
    for category in categories:
        print(f"Current category: {category}")

        try:
            meta_csv = os.path.join(base_path, f"{category}_meta.csv")
            review_csv = os.path.join(base_path, f"{category}_review.csv")

            if not os.path.exists(review_csv) or not os.path.exists(meta_csv):
                print(f"{category} files are missing, skipping")
                continue
            
            try:
                review_data = pd.read_csv(review_csv)
                meta_data = pd.read_csv(meta_csv)

                #Testing
                print("Review data columns before: ", review_data.columns.tolist())
                print("Review meta columns before: ", meta_data.columns.tolist())

                # Merge on parent asin
                if "parent_asin" in review_data.columns and "parent_asin" in meta_data.columns:
                    merged_data = pd.merge(review_data, meta_data, on="parent_asin", how= "inner")
                
                    print("Merged columns after: ", list(merged_data.columns))
                    print(f"Successful merging of {category}")

                    # Handle Invalid / Missing Values

                    #Testing
                    #Before Handle Invalid / Missing Values
                    
                    print("Before")
                    print("Number of rows:", merged_data.shape[0])
                    print("Number of rows where star rating is missing or not in [1–5]:", merged_data[~merged_data["rating"].isin([1.0, 2.0, 3.0, 4.0, 5.0])].shape[0])
                    print("Number of rows where text (the review body) is empty:", merged_data[merged_data["text"].isna()|(merged_data["text"].str.strip() == "")].shape[0])
                    print("Number of rows where brand is unknown:", merged_data.apply(lambda row: True if isinstance(row.get("details"), dict) and "brand" not in row["details"] and (pd.isna(row.get("store")) or row["store"].strip() == "")
                                                          else False, axis=1).sum())
                    print("\n") 
                    
            
                    # Drop rows where star rating is missing or not in [1–5].
                    if "rating" in merged_data.columns:
                        merged_data = merged_data[merged_data["rating"].isin([1.0, 2.0, 3.0, 4.0, 5.0])] #only keep where these are present
            
                    # Drop rows if text (the review body) is empty.
                    if "text" in merged_data.columns:
                        merged_data = merged_data[merged_data["text"].notna()] #dropping nulls
                        merged_data = merged_data[merged_data["text"].str.strip() != ""] #if the space had " " dropping that also
            
                    # If brand cannot be found in the metadata (e.g., missing in details or store), set brand = “Unknown”.
                    if "details" in merged_data.columns or "store" in merged_data.columns:
                        merged_data["brand"] = merged_data.apply(lambda row: 
                                                         (row["details"]["brand"]
                                                          if isinstance(row.get("details"), dict) and "brand" in row["details"]
                                                          else row["store"] 
                                                          if pd.notna(row.get("store")) and row["store"].strip()!= ""
                                                          else "Unknown"), 
                                                          axis=1)
            
            
                    #Testing
                    #After Handle Invalid / Missing Values
                    
                    print("After")
                    print("Number of rows:", merged_data.shape[0])
                    print("Number of rows where star rating is missing or not in [1–5]:", merged_data[~merged_data["rating"].isin([1.0, 2.0, 3.0, 4.0, 5.0])].shape[0])
                    print("Number of rows where text (the review body) is empty:", merged_data[merged_data["text"].isna()|(merged_data["text"].str.strip() == "")].shape[0])
                    print("Number of rows where brand is unknown:", merged_data.apply(lambda row: True if isinstance(row.get("details"), dict) and "brand" not in row["details"] and (pd.isna(row.get("store")) or row["store"].strip() == "")
                                                          else False, axis=1).sum())
                    print("\n")
                    

                    #Remove Duplicates
                    
                    print("Before removing duplicates")
                    print("Number of rows:", merged_data.shape[0])
                    print("\n")
                    
                    merged_data = merged_data.drop_duplicates(subset=["user_id", "asin", "text"], keep= "first")

                    print("After removing duplicates")
                    print("Number of rows:", merged_data.shape[0])
                    print("\n")

                    #Derived Columns

                    #Review Length 
                    if "text" in merged_data.columns:
                        merged_data["review_length"] = merged_data["text"].apply(lambda value: len(re.findall(r'\b\w+\b', value)))

                        #Testing
                        print(merged_data.head(2))

                    #Year
                    #merged_data["timestamp"] = pd.to_datetime(merged_data["timestamp"], unit="ms") 
                    if "timestamp" in merged_data.columns:
                        merged_data["year"] = pd.to_datetime(merged_data["timestamp"], unit="ms").dt.year

                        #Testing
                        print(merged_data[["timestamp", "year"]].head(2))

                    print("Before concatenation")
                    print(f"{category} dataset: ", merged_data.shape)
                    print("\n")

                    files.append(merged_data)       
                else:
                    print(f"No parent_asin in {category}, skipping")

            except Exception as e:
                print(f"Could not process {category}: {e}")

        except Exception as e:
            print(f"Could not process {category}: {e}")
        
    if files:

        #Unified Output
        cleaned_data= pd.concat(files, ignore_index=True)
        print("All categories merging are completed, saving to the csv now...")

        #saving locally
        cleaned_data.to_csv(save_path, index=False)
        print("The fill dataset was added to the csv")

        #Testing
        print("After concatenation")
        print("Full dataset: ", cleaned_data.shape)

        return cleaned_data
    else:
        print("No data was cleaned and merged")
        return None


In [51]:
data_cleaning_and_preprocessing(categories= [
    "All_Beauty", "Amazon_Fashion", "Appliances", "Arts_Crafts_and_Sewing", "Automotive",
    "Baby_Products", "Beauty_and_Personal_Care", "Books", "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories", "Clothing_Shoes_and_Jewelry", "Digital_Music", "Electronics",
    "Gift_Cards", "Grocery_and_Gourmet_Food", "Handmade_Products", "Health_and_Household",
    "Health_and_Personal_Care", "Home_and_Kitchen", "Industrial_and_Scientific", "Kindle_Store",
    "Magazine_Subscriptions", "Movies_and_TV", "Musical_Instruments", "Office_Products",
    "Patio_Lawn_and_Garden", "Pet_Supplies", "Software", "Sports_and_Outdoors",
    "Subscription_Boxes", "Tools_and_Home_Improvement", "Toys_and_Games", "Video_Games", "Unknown"
])

Current category: All_Beauty
Review data columns before:  ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']
Review meta columns before:  ['main_category', 'title', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images', 'videos', 'store', 'categories', 'details', 'parent_asin', 'bought_together', 'subtitle', 'author']
Merged columns after:  ['rating', 'title_x', 'text', 'images_x', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase', 'main_category', 'title_y', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images_y', 'videos', 'store', 'categories', 'details', 'bought_together', 'subtitle', 'author']
Successful merging of All_Beauty
Before
Number of rows: 1034
Number of rows where star rating is missing or not in [1–5]: 0
Number of rows where text (the review body) is empty: 0
Number of rows where brand is unknown: 0


After
Number of

C:\Users\katoy\AppData\Local\Temp\ipykernel_13348\92506054.py:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cleaned_data= pd.concat(files, ignore_index=True)


The fill dataset was added to the csv
After concatenation
Full dataset:  (50506, 28)


,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,videos,store,categories,details,bought_together,subtitle,author,brand,review_length,year
0,3.0,Meh,These were lightweight and soft but much too s...,"[{'attachment_type': 'IMAGE', 'large_image_url...",B088SZDGXG,B08BBQ29N5,AEYORY2AVPMCPDV57CE337YU5LXA,1634275259292,0,True,...,"{'title': ['She loved them !', 'NEW FAUX 24 in...",Niseyo,[],"{""Brand"": ""Niseyo"", ""Extension Length"": ""24 In...",NaN,NaN,NaN,Niseyo,32,2021
1,5.0,Great all natural ear swabs!,I really like these ear swabs. First they come...,[],B07KG1TWP5,B07KG1TWP5,AFSKPY37N3C43SOI5IEXEK5JSIYA,1596473351088,0,False,...,"{'title': [], 'url': [], 'user_id': []}",Bali Boo,[],"{""Is Discontinued By Manufacturer"": ""No"", ""Pac...",NaN,NaN,NaN,Bali Boo,58,2020
2,5.0,Wonderful overnight cream!,"To be honest, I rarely have used an overnight ...",[],B07W397QG4,B07W397QG4,AFSKPY37N3C43SOI5IEXEK5JSIYA,1593352422858,1,False,...,"{'title': ['Great night cream!', 'Iryasa Night...",IRYASA,[],"{""Brand"": ""IRYASA"", ""Item Form"": ""Cream"", ""Uni...",NaN,NaN,NaN,IRYASA,126,2020
3,3.0,just ok,its OK not as good as the original Wet Brush...,[],B077SRDVG9,B077SRDVG9,AHGAOIZVODNHYMNCBV4DECZH42UQ,1522091824726,0,True,...,"{'title': [], 'url': [], 'user_id': []}",Fine Touch,[],"{""Brand"": ""Fine Touch"", ""Shape"": ""Oblong"", ""Un...",NaN,NaN,NaN,Fine Touch,14,2018
4,5.0,clip rollers,I purchased these to see if I could set my mom...,[],B00946GD7K,B00946HGLW,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1596901440832,0,True,...,"{'title': ['Diane Magnetic Hair Rollers', 'How...",Annie,[],"{""Hair Type"": ""Curly"", ""Number of Items"": ""1"",...",NaN,NaN,NaN,Annie,53,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50501,5.0,Awesome,This is really big the color is bright and bold.,[],B08ZMXRZJX,B08ZMXRZJX,AHHFW36BP4VMQWC6V2NTKIXFAA2A,1656605455178,0,True,...,"{'title': ['Colorful tapestry,clear print,many...",GaGoWa,[],"{""Wall Art Form"": ""Tapestry"", ""Material"": ""Pol...",NaN,NaN,NaN,GaGoWa,10,2022
50502,5.0,Price,Price,[],B079TBDDQY,B079TBDDQY,AGA3PQKDGXZQB7D3XXO74OO3GM7A,1604071140702,1,True,...,"{'title': [], 'url': [], 'user_id': []}",Pepsi,[],"{""Brand"": ""Pepsi"", ""Item Form"": ""Liquid"", ""Fla...",NaN,NaN,NaN,Pepsi,1,2020
50503,5.0,Good tasting,Never had white macaroni and cheese something ...,[],B017XKOT6A,B017XKOT6A,AHEJ5LC7BSEADCIZQQQPZPVWOLCA,1596401128194,0,True,...,"{'title': [], 'url': [], 'user_id': []}",Annie's Homegrown,[],"{""Brand"": ""Annie's Homegrown"", ""Number of Item...",NaN,NaN,NaN,Annie's Homegrown,8,2020
50504,5.0,Great snack,"These are so good, and so addicting. I have pr...",[],B0017KYJK4,B0017KYJ68,AFOGWJCZMC626A5IBK32HEHZB4VA,1435871356000,0,True,...,"{'title': [""Kar's Nuts Variety Offerings "", ""R...",Kar's,[],"{""Brand"": ""Kar's"", ""Flavor"": ""Peanut Butter, C...",NaN,NaN,NaN,Kar's,34,2015
